<a href="https://colab.research.google.com/github/hscrown/madatpublicdata/blob/main/%EC%82%AC%EC%9A%A9%EC%9E%90_%ED%8F%89%EC%A0%90%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%A7%8C%EB%93%A4%EA%B8%B0_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install datasets

from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from datasets import load_dataset

## 데이터로드 및 전처리

도서관데이터

In [12]:
# 필요한 컬럼 # 시설명, 구이름, 장소구분
lib = pd.read_csv('seoul_libs.csv', encoding='utf-8')
lib.head()

# 밸류값이 모두 'lib'인 ['type'] 칼럼 추가
lib['type'] = 'lib'

# 컬럼명 변경
cols= {'도서관 일련번호':'place_id', '도서관명':'place_name', '구명':'gu_name'}
lib = lib.rename(columns=cols)
lib.columns

# # 필요한 컬럼만 추출
lib=lib[['place_name', 'gu_name', 'type']]
lib.head()


,place_name,gu_name,type
0,BIBLIOTECA,강남구,lib
1,LH강남3단지작은도서관,강남구,lib
2,休+밤토리SH작은도서관,강남구,lib
3,강남구립못골도서관,강남구,lib
4,강남데시앙파크작은도서관,강남구,lib


공원데이터

In [13]:
# 필요한 컬럼 # 시설명, 구이름, 장소구분
par = pd.read_csv('seoul_parks.csv', encoding='utf-8')
par.head()
par.columns
# 밸류값이 모두 'par'인 ['type'] 칼럼 추가
par['type'] = 'par'

# 컬럼명 변경
cols= {'공원명':'place_name', '공원주소':'gu_name'}
par = par.rename(columns=cols)
par.columns

# 공원주소 컬럼에서 구 이름만 추출
par['gu_name'] = par['gu_name'].str.split(' ', expand=True)[1]

# 결측치 삭제
par = par.dropna()

# gu_name이 '구'로 끝나지 않으면 삭제
par = par[par['gu_name'].str.endswith('구')]

# # 필요한 컬럼만 추출
par=par[['place_name', 'gu_name', 'type']]
par.head()

,place_name,gu_name,type
0,남산도시자연공원,중구,par
12,여의도근린공원,영등포구,par
16,보라매근린공원,동작구,par
17,어린이대공원,광진구,par
18,천호근린공원,강동구,par


박물관, 미술관 데이터

In [14]:
dataset = load_dataset("hscrown/seoul_museums")
muse = pd.DataFrame(dataset['train'])

muse.columns

# 밸류값이 모두 'muse'인 ['type'] 칼럼 추가
muse['type'] = 'muse'

# 컬럼명 변경
cols= {'시설명':'place_name', '주소':'gu_name'}
muse = muse.rename(columns=cols)

# 주소 컬럼에서 구 이름만 추출
muse['gu_name'] = muse['gu_name'].str.split(' ', expand=True)[1]

# gu_name이 '구'로 끝나지 않으면 삭제
muse = muse[muse['gu_name'].str.endswith('구')]
# # 필요한 컬럼만 추출
muse=muse[['place_name', 'gu_name', 'type']]

muse.tail()

Repo card metadata block was not found. Setting CardData to empty.


,place_name,gu_name,type
170,매헌윤봉길의사기념관,서초구,muse
171,동덕여자대학교 박물관,성북구,muse
172,DDP디자인뮤지엄,중구,muse
173,아모레퍼시픽미술관,용산구,muse
174,롯데뮤지엄,송파구,muse


In [15]:
# 공원데이터와 도서관 데이터 합침
df = pd.concat([lib, par,muse])
df.tail()

# 인덱스 초기화
df = df.reset_index(drop=True)

# 'place_id' 컬럼 만들고 인덱스 밸류로 넣기
df['place_id'] = df.index

df.head()

# df 를 csv 파일로 변환
df.to_csv('places.csv', index=False, encoding='utf-8')

In [16]:
df.tail()

,place_name,gu_name,type,place_id
1658,매헌윤봉길의사기념관,서초구,muse,1658
1659,동덕여자대학교 박물관,성북구,muse,1659
1660,DDP디자인뮤지엄,중구,muse,1660
1661,아모레퍼시픽미술관,용산구,muse,1661
1662,롯데뮤지엄,송파구,muse,1662


# 사용자 평점데이터 만들기

In [19]:
import pandas as pd
import numpy as np

def generate_user_ratings(num_users):
    users = [df.copy() for _ in range(num_users)]

    # user_id를 각각 가지도록 열 이름을 바꾸자
    for user_id, user_df in enumerate(users):
        user_df.columns = ['place_name','user_id', 'rating','place_id',]
        user_df['user_id'] = user_id

    # 사용자 데이터프레임을 합치자
    user_rating = pd.concat(users)

    # 'rating' 열의 NaN 값을 0으로 채워
    user_rating['rating'] = user_rating['rating'].fillna(0)

    # 평점을 평균이 3.5이고 표준 편차가 1.5인 분포로 나오게해서 정규분포와 비슷하면서 0값이 나오게함.
    ratings_mean = 3.5
    ratings_std = 1.5
    raw_ratings = np.random.normal(ratings_mean, ratings_std, size=len(user_rating))

    # 평점을 가장 가까운 0.5로 반올림해
    rounded_ratings = np.round(raw_ratings * 2) / 2

    # 평점을 0과 5 사이로 맞춰
    rounded_ratings = np.clip(rounded_ratings, 0, 5)

    user_rating['rating'] = rounded_ratings

    # 각 사용자의 평균 평점을 계산합니다.
    user_means = user_rating.groupby('user_id')['rating'].mean()

    # 각 사용자의 평균 평점을 중심으로 노이즈를 추가합니다.
    for user_id, mean_rating in user_means.items():
        # 해당 사용자의 데이터 인덱스를 가져옵니다.
        user_indices = user_rating['user_id'] == user_id
        # 평균 평점을 중심으로 표준 정규 분포에서 샘플링하여 노이즈를 추가합니다.
        noise = np.random.normal(loc=0, scale=1, size=np.sum(user_indices))
        # 원래의 평점에 노이즈를 더합니다.
        user_rating.loc[user_indices, 'rating'] += noise

    # 평점을 0과 5 사이로 맞춥니다.
    user_rating['rating'] = user_rating['rating'].clip(0, 5)

    # 평점을 가장 가까운 0.5로 반올림해
    rounded_ratings = np.round(user_rating['rating'] * 2) / 2

    # 소수점 1의 자리에서 잘라
    rounded_ratings = np.clip(rounded_ratings, 0, 5)

    user_rating['rating'] = rounded_ratings

    # place_id와 place_name을 가진데이터
    id_match = df[['place_id', 'place_name']]

    # user_rating은 place_name 열삭제
    user_rating = user_rating.drop('place_name', axis=1)



    return id_match,user_rating

# 사용자 수를 바꿔서 테스트해봐
num_users = 1000
id_match, user_rating = generate_user_ratings(num_users)
user_rating.tail()


,user_id,rating,place_id
1658,999,5.0,1658
1659,999,4.5,1659
1660,999,5.0,1660
1661,999,3.5,1661
1662,999,2.0,1662


In [20]:
user_rating.to_csv(f'user_rating_{num_users}.csv', index=False, encoding='utf-8')
id_match.to_csv(f'id_match_{num_users}.csv', index=False, encoding='utf-8')
